# 1. GPU 설정

- Google Colaboratory에서 딥러닝 학습을 진행하실 때에는 반드시 GPU 설정이 제대로 되어있는지 확인해야 합니다.
- 위의 메뉴에서 '런타임 - 런타임 유형 변경 - 하드웨어 가속기'에서 GPU를 선택해줍니다.

# 2. Pytorch 설치

- Google Colaboratory에서 최근 들어 Pytorch도 기본적으로 제공을 하기 시작해서 이 과정은 생략해도 됩니다. 
- 런타임 최초 연결시 PIL(Python Image Library)에서 오류가 발생할 수 있으니, 오류가 발생한 경우 런타임을 다시 연결해주면 해결됩니다.

In [1]:
!pip install torch
!pip install torchvision

    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ahn92\AppData\Local\Temp\pip-install-rqqpodfw\torch\setup.py", line 11, in <module>
        raise RuntimeError(README)
    RuntimeError: PyTorch does not currently provide packages for PyPI (see status at https://github.com/pytorch/pytorch/issues/566).
    
    Please follow the instructions at http://pytorch.org/ to install with miniconda instead.
    
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\ahn92\AppData\Local\Temp\pip-install-rqqpodfw\torch\
You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/5f/e9/bac4204fe9cb1a002ec6140b47f51affda1655379fe302a1caef421f9846/torch-0.1.2.post1.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ahn92\AppData\Local\Temp\pip-install-xhi55cln\torch\setup.py", line 11, in <module>
        raise RuntimeError(README)
    RuntimeError: PyTorch does not currently provide packages for PyPI (see status at https://github.com/pytorch/pytorch/issues/566).
    
    Please follow the instructions at http://pytorch.org/ to install with miniconda instead.
    
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\ahn92\AppData\Local\Temp\pip-install-xhi55cln\torch\
You are using pip version 18.1, however version 19.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# 3. Import

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np

# 4. Data Loader

- Pytorch의 데이터 로더는 크게 __'Dataset'__과 __'Data Loader'__로 구분되어 있습니다.
- 1) __Dataset__ : 학습에 사용한 데이터셋 전체를 의미합니다. <br>  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Dataset의 파라미터 중 하나인 __transform__을 통해 전체 데이터에 일괄적으로 변형을 줄 수 있습니다.
<br> 
- 2) __Data Loader__ : Dataset에서 데이터를 읽어와 모델에 입력으로 제공해주는역할을 합니다. 

이번 실습에는 __CIFAR 10__이라는 데이터를 사용하도록 하겠습니다.
<br>CIFAR 10은 Pytorch에서 기본적으로 제공하는 데이터셋이기 때문에 별도로 데이터를 다운로드 해줄 필요가 없습니다.

![CIFAR_10](http://solarisailab.com/wp-content/uploads/2017/06/cifar-10_labels.png)

In [0]:
# Transform

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

### 1) Train

In [0]:
# Train Dataset 선언

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

In [0]:
# Train Data Loader 선언

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

### 2) Test

In [6]:
# Test Dataset 선언 (Train과 동일한 transform 사용)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [0]:
# Test Data Loader 선언

testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

### 3) CIFAR 10의 클래스

In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# 5. 모델 (네트워크)

- 이번 실습에는 Classification 모델 중 하나인 __'Resnet18'__을 사용하도록 하겠습니다.
- 마찬가지로 Pytorch는 많은 딥러닝 모델도 기본적으로 제공하기 때문에 코드 한 줄로 편리하게 모델을 불러올 수 있습니다.

In [0]:
# CNN 모델 선언

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [0]:
model = Net()

# 6. 손실함수(Loss function)과 Optimizer

- Multi-label Classification에서 가장 흔히 사용하는 loss 중 하나인 __'Cross Entropy Loss'__를 사용하도록 하겠습니다.
- 그리고 Optimizer는 주로 __'Adam'__과 __'SGD (Stochastic Gradient Descent)'__가 사용되고 이번 실습에는 __SGD__를 사용하겠습니다.

In [0]:
# Loss function 선언

criterion = nn.CrossEntropyLoss()

In [0]:
# Optimizer SGD 선언

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 7. GPU in Pytorch

- Pytorch에서 GPU를 사용하려면 __모델과 데이터를 GPU 설정(Cuda)으로 바꿔주어야 합니다.__

In [13]:
# GPU 사용이 가능한 환경인지 확인

print(torch.cuda.is_available())

True


In [14]:
# GPU 사용 가능 여부를 바탕으로 'GPU(cuda)'와 'CPU' 중 어떤 것을 바탕으로 학습을 할지 설정

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [0]:
# 모델 GPU 설정

model.to(device)

# 8. 학습

![Term](https://image.slidesharecdn.com/random-171120113837/95/-43-638.jpg?cb=1511177981)
<br>※ https://www.slideshare.net/w0ong/ss-82372826

- __Epoch__ : 전체 데이터셋을 학습하는 횟수
- __Batch Size__ : 데이터셋을 쪼갠 크기
- __Iteration (= Step)__ : Batch size 단위로 학습한 횟수

In [0]:
# 전체 데이터셋을 몇 번 학습할 지 설정

epochs = 2

In [0]:
# 학습 진행

for epoch in range(epochs):
    print('\n===> Epoch [%d/%d]' % (epoch+1, epochs))
    
    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
      
        # Data Load
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward
        loss.backward()
        
        # Optimization
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        
        if i % 2000 == 1999:    # Print every 2000 mini-batches
            print('      - Iteration [%5d / %5d] --- Loss: %.3f' %
                  (i+1, len(trainloader), running_loss / 2000))
            running_loss = 0.0

# 9. Test

In [0]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))


with torch.no_grad(): # Freeze model
  
    for data in testloader:
        
        # Data Load
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        # Predict
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        # Calculuate
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))